In [1]:
''''Qn1

Write a python program which searches all the product under a particular product from www.amazon.in. The 
product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for 
guitars'''

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd


path_chromedriver = r'C:\chromedriver-win64\chromedriver.exe'  # path to chromedriver executable
service = Service(path_chromedriver)
driver = webdriver.Chrome(service=service)

# user input for product to search
product_name = input("Enter the product to search: ")

# Open Amazon
driver.get('https://www.amazon.in')


time.sleep(3)

# search bar and enter the product name
search_box = driver.find_element(By.ID, "twotabsearchtextbox")
search_box.send_keys(product_name)
search_box.send_keys(Keys.RETURN)


time.sleep(3)

# Extract product det
products = []
try:
   
    product_containers = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

    for container in product_containers:
        # product name
        try:
            name = container.find_element(By.XPATH, './/h2/a/span').text
        except:
            name = '-'
        
        # product price
        try:
            price = container.find_element(By.XPATH, './/span[@class="a-price-whole"]').text
        except:
            price = '-'

        # product rating
        try:
            rating = container.find_element(By.XPATH, './/span[@class="a-icon-alt"]').text
        except:
            rating = '-'

        # Append product details to the list
        products.append({
            'Product Name': name,
            'Price': price,
            'Rating': rating
        })
finally:
    driver.quit()

# Convert to DataFrame 
df = pd.DataFrame(products)
df.replace('', '-', inplace=True)  # Replace empty values with hyphens
print(df)

# save to CSV
df.to_csv('amazon_products.csv', index=False)


Enter the product to search:  television


                                         Product Name   Price Rating
0   Xiaomi 125 cm (50 inches) X Series 4K LED Smar...  33,999      -
1   Xiaomi 138 cm (55 inches) X Pro QLED Series Sm...  49,999      -
2   Redmi 80 cm (32 inches) F Series HD Ready Smar...  10,999      -
3   VW 80 cm (32 inches) Frameless Series HD Ready...   7,499      -
4   Kodak 80 cm (32 inches) Special Edition Series...   7,999      -
5   Redmi Xiaomi 108 cm (43 inches) 4K Ultra HD Sm...  19,990      -
6   TCL 101 cm (40 inches) Mettalic Bezel-Less Ful...       -      -
7   Samsung 80 cm (32 inches) HD Ready Smart LED T...  15,240      -
8   TCL 80.04 cm (32 inches) Metallic Bezel-Less H...  10,990      -
9   Xiaomi Smart TV A 80 cm (32) HD Ready Smart Go...  14,489      -
10  Acer 80 cm (32 inches) V Series HD Ready Smart...  13,999      -
11  Wobble 139.7 cm (55 inches) UD Series 4K Ultra...  30,999      -
12  LG 80 cm (32 inches) HD Ready Smart LED TV 32L...  14,490      -
13  LG 80 cm (32 inches) HD Ready 

In [2]:
'''Qn2

In the above question, now scrape the following details of each product listed in first 3 pages of your search 
results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then 
scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.
'''

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd


path_chromedriver = r'C:\chromedriver-win64\chromedriver.exe'  # path to chromedriver executable
service = Service(path_chromedriver)
driver = webdriver.Chrome(service=service)



# user input for product to search
product_name = input("Enter the product to search: ")

# Open Amazon.in 
driver.get('https://www.amazon.in')

# Wait for the page to load completely
time.sleep(3)

# Land enter the product name
search_box = driver.find_element(By.ID, "twotabsearchtextbox")
search_box.send_keys(product_name)
search_box.send_keys(Keys.RETURN)

# Wait for the search results to load
time.sleep(3)

# Function to scrape details from the current page
def scrape_page():
    products = []
    
    
    product_containers = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

    for container in product_containers:
        # Extract product name
        try:
            product_name = container.find_element(By.XPATH, './/h2/a/span').text
        except:
            product_name = '-'
        
        # Extract product URL
        try:
            product_url = container.find_element(By.XPATH, './/h2/a').get_attribute('href')
        except:
            product_url = '-'

        # Extract brand name (it may sometimes be part of product name)
        try:
            brand_name = container.find_element(By.XPATH, './/h5/span').text
        except:
            brand_name = '-'

        # Extract product price
        try:
            price = container.find_element(By.XPATH, './/span[@class="a-price-whole"]').text
        except:
            price = '-'

        # return/exchange policy 
        try:
            return_exchange = container.find_element(By.XPATH, './/span[contains(text(),"Return")]').text
        except:
            return_exchange = '-'

        # expected delivery date
        try:
            expected_delivery = container.find_element(By.XPATH, './/span[contains(text(),"Get it by")]').text
        except:
            expected_delivery = '-'

        # availability status 
        try:
            availability = container.find_element(By.XPATH, './/span[contains(text(),"In stock")]').text
        except:
            availability = '-'

        # Append product details to the list
        products.append({
            'Brand Name': brand_name,
            'Product Name': product_name,
            'Price': price,
            'Return/Exchange': return_exchange,
            'Expected Delivery': expected_delivery,
            'Availability': availability,
            'Product URL': product_url
        })
    
    return products

# Initialize list 
all_products = []

# Scrape products from the first 3 pages 
for page in range(1, 4):
    print(f"Scraping page {page}...")
    
    
    all_products.extend(scrape_page())
    
    
    try:
        next_button = driver.find_element(By.XPATH, '//a[contains(@class, "s-pagination-next")]')
        next_button.click()
        time.sleep(3) 
    except:
        print("No more pages found.")
        break


driver.quit()

# Convert the product data into a pandas DataFrame
df = pd.DataFrame(all_products)
df.replace('', '-', inplace=True)  # Replace any empty values with hyphens

# Save the data to CSV file
df.to_csv('amazon_products.csv', index=False)


print(df)


Enter the product to search:  television


Scraping page 1...
Scraping page 2...
Scraping page 3...
   Brand Name                                       Product Name     Price  \
0           -  Xiaomi 125 cm (50 inches) X Series 4K LED Smar...    33,999   
1           -  Xiaomi 138 cm (55 inches) X Pro QLED Series Sm...    49,999   
2           -  Redmi 80 cm (32 inches) F Series HD Ready Smar...    10,999   
3           -  VW 80 cm (32 inches) Frameless Series HD Ready...     7,499   
4           -  Kodak 80 cm (32 inches) Special Edition Series...     7,999   
..        ...                                                ...       ...   
61          -  LG 139 cm (55 inches) 4K Ultra HD Smart OLED T...  1,04,990   
62          -  TOSHIBA 189 cm (75 inches) C350NP Series 4K Ul...    82,999   
63          -  Sony Bravia 164 cm (65 inches) 4K Ultra HD Sma...    77,990   
64          -  MI 108 cm (43 inches) A Series Full HD Smart G...    21,999   
65          -  Xiaomi 108 cm (43 inches) X Series 4K LED Smar...    26,999   

   Ret

In [3]:
'''Qn3
Write a python program to access the search bar and search button on images.google.com and scrape 10 
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’. 
'''
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import requests
import os


path_chromedriver = r'C:\chromedriver-win64\chromedriver.exe'  # path to chromedriver executable
service = Service(path_chromedriver)
driver = webdriver.Chrome(service=service)

# Keywords to search for
keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

# scrape images for a specific keyword
def scrape_images(keyword, num_images=10):
    print(f"Searching for {keyword}...")
    
    
    driver.get('https://images.google.com')
    time.sleep(2)
    
    # search bar and enter the keyword
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.RETURN)
    
  
    time.sleep(2)

   
    for _ in range(3):
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
        time.sleep(2)

    # image URLs
    image_urls = []
    image_elements = driver.find_elements(By.XPATH, '//img[@class="rg_i Q4LuWd"]')

    for image in image_elements[:num_images]:
        try:
            
            image.click()
            time.sleep(2)
            large_image = driver.find_element(By.XPATH, '//img[@class="n3VNCb"]')
            image_url = large_image.get_attribute('src')
            if 'http' in image_url:  # Only add valid image URLs
                image_urls.append(image_url)
        except:
            continue

    return image_urls

# download images from URLs
def download_images(image_urls, keyword):
    # Create a directory for the keyword
    if not os.path.exists(keyword):
        os.makedirs(keyword)
    
    # Download and save images
    for i, url in enumerate(image_urls):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                with open(f"{keyword}/{keyword}_{i+1}.jpg", 'wb') as file:
                    file.write(response.content)
            else:
                print(f"Failed to download image {i+1} from {url}")
        except Exception as e:
            print(f"Error downloading image {i+1}: {e}")

# Scrape and download images 
for keyword in keywords:
    image_urls = scrape_images(keyword, num_images=10)
    download_images(image_urls, keyword)


driver.quit()

print("Image scraping and downloading completed!")


Searching for fruits...
Searching for cars...
Searching for Machine Learning...
Searching for Guitar...
Searching for Cakes...
Image scraping and downloading completed!


In [ ]:
'''Qn4:
Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com
and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand 
Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV. 
'''
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd


path_chromedriver = r'C:\chromedriver-win64\chromedriver.exe'  # path to chromedriver executable
service = Service(path_chromedriver)
driver = webdriver.Chrome(service=service)

# Get the smartphone to search from user input
smartphone_name = input("Enter the smartphone name to search: ")


driver.get('https://www.flipkart.com')
time.sleep(3)

# Close the login popup if appears
try:
    close_popup = driver.find_element(By.XPATH, '//button[contains(text(),"✕")]')
    close_popup.click()
except:
    pass

# enter the smartphone name
search_box = driver.find_element(By.NAME, 'q')
search_box.send_keys(smartphone_name)
search_box.send_keys(Keys.RETURN)


time.sleep(3)

# Function to scrape details search results
def scrape_smartphones():
    smartphones = []
    
  
    product_containers = driver.find_elements(By.XPATH, '//div[@class="_1AtVbE"]//div[@class="_13oc-S"]')

    for container in product_containers:
        # Extract brand and smartphone name
        try:
            smartphone_name = container.find_element(By.XPATH, './/div[@class="_4rR01T"]').text
            brand_name = smartphone_name.split()[0]  # First word is usually the brand
        except:
            smartphone_name = '-'
            brand_name = '-'

        # Extract product URL
        try:
            product_url = container.find_element(By.XPATH, './/a').get_attribute('href')
        except:
            product_url = '-'

        # Extract price
        try:
            price = container.find_element(By.XPATH, './/div[@class="_30jeq3 _1_WHN1"]').text
        except:
            price = '-'

        # Extract other specifications
        try:
            specs = container.find_element(By.XPATH, './/ul[@class="_1xgFaf"]').text.split('\n')
        except:
            specs = []

        
        color, ram, rom, primary_camera, secondary_camera, display_size, battery_capacity = '-', '-', '-', '-', '-', '-', '-'

        
        for spec in specs:
            if 'RAM' in spec and 'ROM' in spec:
                ram = spec.split('|')[0].strip()
                rom = spec.split('|')[1].strip()
            elif 'Display' in spec:
                display_size = spec
            elif 'Battery' in spec:
                battery_capacity = spec
            elif 'Camera' in spec and 'Primary' in spec:
                primary_camera = spec
            elif 'Camera' in spec and 'Secondary' in spec:
                secondary_camera = spec
            elif 'Color' in spec:
                color = spec

        # Append the smartphone details to the list
        smartphones.append({
            'Brand Name': brand_name,
            'Smartphone Name': smartphone_name,
            'Colour': color,
            'RAM': ram,
            'Storage (ROM)': rom,
            'Primary Camera': primary_camera,
            'Secondary Camera': secondary_camera,
            'Display Size': display_size,
            'Battery Capacity': battery_capacity,
            'Price': price,
            'Product URL': product_url
        })
    
    return smartphones

# Scrape the smartphone data
smartphone_data = scrape_smartphones()


driver.quit()

# Convert the data into a pandas DataFrame
df = pd.DataFrame(smartphone_data)
df.replace('', '-', inplace=True)  # Replace any empty values with hyphens

# Save the DataFrame to a CSV file
df.to_csv('flipkart_smartphones.csv', index=False)


print(df)


In [5]:
'''
Qn5:

Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

'''
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time

# Set up the Chrome webdriver service
path_chromedriver = r'C:\chromedriver-win64\chromedriver.exe'  # path to chromedriver executable
service = Service(path_chromedriver)
driver = webdriver.Chrome(service=service)

# Get the city name 
city_name = input("Enter the city name: ")

# Open Google Maps
driver.get('https://www.google.com/maps')
time.sleep(3)  

# Find the search bar and enter the city name
search_box = driver.find_element(By.ID, "searchboxinput")
search_box.send_keys(city_name)
search_box.send_keys(Keys.RETURN)


time.sleep(5)


current_url = driver.current_url

# Close browser
driver.quit()

# Extract latitude and longitude from the URL
try:
   
    url_parts = current_url.split('@')[1].split(',')
    latitude = url_parts[0]
    longitude = url_parts[1]
    
    print(f"Coordinates of {city_name}:")
    print(f"Latitude: {latitude}")
    print(f"Longitude: {longitude}")
except Exception as e:
    print(f"Could not extract coordinates: {e}")


Enter the city name:  CHENNAI


Coordinates of CHENNAI:
Latitude: 13.0473168
Longitude: 79.8793111


In [ ]:
'''
Qn6:
Write a program to scrap all the available details of best gaming laptops from digit.in. 
'''
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd


path_chromedriver = r'C:\chromedriver-win64\chromedriver.exe'  # path to chromedriver executable
service = Service(path_chromedriver)
driver = webdriver.Chrome(service=service)

# URL of the best gaming laptops page on digit.in
url = "https://www.digit.in/top-products/best-gaming-laptops-40.html"


driver.get(url)


time.sleep(5)

# Initialize an empty list to store the laptop details
laptop_data = []

# Find all laptop sections on the page
laptops = driver.find_elements(By.CSS_SELECTOR, "div.TopNumbeHeading")

for laptop in laptops:
    # Extract the laptop 
    try:
        laptop_name = laptop.find_element(By.TAG_NAME, "h2").text.strip()
    except:
        laptop_name = '-'
    
    # Extract price
    try:
        price = laptop.find_element(By.CLASS_NAME, "price").text.strip()
    except:
        price = '-'

    # Extract specifications
    try:
        specs = laptop.find_element(By.CLASS_NAME, "Section-center").text.strip().split('\n')
        specs = [spec.strip() for spec in specs if spec.strip()]
    except:
        specs = []

    # Create dictionary for laptop details
    laptop_details = {
        "Laptop Name": laptop_name,
        "Price": price,
        "Specifications": "; ".join(specs) if specs else '-'
    }

    # Append laptop details to the list
    laptop_data.append(laptop_details)


driver.quit()

# Convert list of laptop data into a pandas DataFrame
df = pd.DataFrame(laptop_data)

# Display DataFrame
print(df)

# Save DataFrame to a CSV file
df.to_csv("best_gaming_laptops_digit.csv", index=False)


In [ ]:
'''
Qn7:
Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: 
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”. 
'''
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time

path_chromedriver = r'C:\chromedriver-win64\chromedriver.exe'  # path to chromedriver executable
service = Service(path_chromedriver)
driver = webdriver.Chrome(service=service)

# URL of Forbes Billionaires list
url = "https://www.forbes.com/billionaires/"


driver.get(url)


time.sleep(5)

# Initialize an empty list 
billionaires_data = []


driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)

# Loop through all billionaire entries
billionaires = driver.find_elements(By.CSS_SELECTOR, "div.personName")

for billionaire in billionaires:
    # Extract rank
    try:
        rank = billionaire.find_element(By.XPATH, ".//div[@class='rank']").text
    except:
        rank = '-'
    
    # Extract name
    try:
        name = billionaire.find_element(By.CSS_SELECTOR, ".personName").text
    except:
        name = '-'
    
    # Extract net worth
    try:
        net_worth = billionaire.find_element(By.CSS_SELECTOR, ".netWorth").text
    except:
        net_worth = '-'
    
    # Extract age
    try:
        age = billionaire.find_element(By.CSS_SELECTOR, ".age").text
    except:
        age = '-'
    
    # Extract citizenship
    try:
        citizenship = billionaire.find_element(By.CSS_SELECTOR, ".countryOfCitizenship").text
    except:
        citizenship = '-'
    
    # Extract source of wealth
    try:
        source = billionaire.find_element(By.CSS_SELECTOR, ".source").text
    except:
        source = '-'
    
    # Extract industry
    try:
        industry = billionaire.find_element(By.CSS_SELECTOR, ".category").text
    except:
        industry = '-'
    
    # Append data to the list
    billionaires_data.append({
        'Rank': rank,
        'Name': name,
        'Net Worth': net_worth,
        'Age': age,
        'Citizenship': citizenship,
        'Source': source,
        'Industry': industry
    })


driver.quit()

# Convert list of data into a pandas DataFrame
df = pd.DataFrame(billionaires_data)

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv("forbes_billionaires.csv", index=False)


In [ ]:
'''
Qn8:
Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video. from selenium import webdriver
'''
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd


path_chromedriver = r'C:\chromedriver-win64\chromedriver.exe'  # path to chromedriver executable
service = Service(path_chromedriver)
driver = webdriver.Chrome(service=service)

# URL of the YouTube video 
youtube_video_url = "https://www.youtube.com/watch?v=GwIo3gDZCVQ"

# Open the YouTube page
driver.get(youtube_video_url)


time.sleep(5)

# Scroll to the comments section
driver.execute_script("window.scrollTo(0, 800);")
time.sleep(3)


num_scrolls = 20  # the number of scrolls 
for _ in range(num_scrolls):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(3)  

# Initialize lists to store data
comments_data = []

# Find all comment sections on the page
comments = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
upvotes = driver.find_elements(By.XPATH, '//*[@id="vote-count-middle"]')
timestamps = driver.find_elements(By.XPATH, '//*[@id="header-author"]/yt-formatted-string/a')

# Extract at least 500 comments
for i in range(min(5, len(comments))):
 try:
    comment_text = comments[i].text
    upvote_count = upvotes[i].text if upvotes[i].text != '' else '0'  # Default to 0 if no upvotes are shown
    timestamp = timestamps[i].text
 except:
    break

    # Append data to the list
    comments_data.append({
        'Comment': comment_text,
        'Upvotes': upvote_count,
        'Timestamp': timestamp

    })


driver.quit()

# Convert list of data into a pandas DataFrame
df = pd.DataFrame(comments_data)

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv("youtube_comments.csv", index=False)


In [ ]:
'''Qn 9:
Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall 
reviews, privates from price, dorms from price, facilities and property description. from selenium import webdriver
'''
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd


path_chromedriver = r'C:\chromedriver-win64\chromedriver.exe'  # path to chromedriver executable
service = Service(path_chromedriver)
driver = webdriver.Chrome(service=service)

# URL of the hostel search page for London
url = "https://www.hostelworld.com/s?q=London&city=London&type=city&id=3&from=2022-12-14&guests=2"

# Open the hostelworld page 
driver.get(url)


time.sleep(5)

# Initialize an empty list 
hostel_data = []

# Scroll and load more hostels 
for _ in range(5):  
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

# Find all the hostel sections on the page
hostels = driver.find_elements(By.CLASS_NAME, 'property')

# Loop through each hostel and extract the details
for hostel in hostels:
    # Extract the hostel name
    try:
        hostel_name = hostel.find_element(By.CLASS_NAME, 'title').text.strip()
    except:
        hostel_name = '-'

    # Extract the distance from city center
    try:
        distance = hostel.find_element(By.CLASS_NAME, 'distance').text.strip()
    except:
        distance = '-'

    # Extract the rating
    try:
        rating = hostel.find_element(By.CLASS_NAME, 'score').text.strip()
    except:
        rating = '-'

    # Extract the total reviews
    try:
        total_reviews = hostel.find_element(By.CLASS_NAME, 'reviews').text.strip()
    except:
        total_reviews = '-'

    # Extract the overall reviews 
    try:
        overall_reviews = hostel.find_element(By.CLASS_NAME, 'keyword').text.strip()
    except:
        overall_reviews = '-'

    # Extract the price 
    try:
        privates_from_price = hostel.find_element(By.CLASS_NAME, 'price-privates').text.strip().replace('\n', ' ')
    except:
        privates_from_price = '-'

    # Extract the price for dorms
    try:
        dorms_from_price = hostel.find_element(By.CLASS_NAME, 'price-dorms').text.strip().replace('\n', ' ')
    except:
        dorms_from_price = '-'

    # Extract the facilities
    try:
        facilities = hostel.find_element(By.CLASS_NAME, 'facilities').text.strip().replace('\n', ', ')
    except:
        facilities = '-'

    # Extract the property description
    try:
        property_description = hostel.find_element(By.CLASS_NAME, 'description').text.strip()
    except:
        property_description = '-'

    # Append the hostel details to the list
    hostel_data.append({
        'Hostel Name': hostel_name,
        'Distance from City Centre': distance,
        'Rating': rating,
        'Total Reviews': total_reviews,
        'Overall Reviews': overall_reviews,
        'Privates From Price': privates_from_price,
        'Dorms From Price': dorms_from_price,
        'Facilities': facilities,
        'Property Description': property_description
    })


driver.quit()

# Convert the list of hostel data into a pandas DataFrame
df = pd.DataFrame(hostel_data)

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv("hostels_in_london.csv", index=False)
